# Assigment: PyTorch Model Zoo

The [PyTorch "Model Zoo"](https://pytorch.org/vision/stable/models.html) provides a large number of pre-trained CNN models and vision [data sets](https://pytorch.org/vision/stable/datasets.html)...

In [1]:
#imports
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
#transform input data (image) to tensor
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#set batch size
batch_size = 4

#First dataloader
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:12<00:00, 13.3MB/s]


## Assignment 1:
Load a "*ResNet18*" from the torchvision model zoo and train it for 10 epochs

In [3]:

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#Load ResNet‑18 vom Zoo ohne vortraining! (DEFAULT)
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
model = model.to(device)

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]


In [4]:
#Loss function + optimiser
criterion = torch.nn.CrossEntropyLoss()
#SGD
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.00005)


In [5]:
#______________________________
#Training (10 epochs)
#______________________________
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / len(trainset)
    epoch_acc = 100.0 * correct / total
    print(f"Epoche [{epoch+1:02d}/{num_epochs}]  "
          f"Train‑loss: {epoch_loss:.4f}  Train‑acc: {epoch_acc:.2f}%")

Epoche [01/10]  Train‑loss: 1.9283  Train‑acc: 35.03%
Epoche [02/10]  Train‑loss: 1.4998  Train‑acc: 48.65%
Epoche [03/10]  Train‑loss: 1.3443  Train‑acc: 55.01%
Epoche [04/10]  Train‑loss: 1.2110  Train‑acc: 59.87%
Epoche [05/10]  Train‑loss: 1.2049  Train‑acc: 60.22%
Epoche [06/10]  Train‑loss: 1.1367  Train‑acc: 62.33%
Epoche [07/10]  Train‑loss: 1.0441  Train‑acc: 65.38%
Epoche [08/10]  Train‑loss: 1.0583  Train‑acc: 65.03%
Epoche [09/10]  Train‑loss: 1.0375  Train‑acc: 65.95%
Epoche [10/10]  Train‑loss: 0.9830  Train‑acc: 67.89%


In [6]:
#______________________________________________________
# Eval
#_______________________________________________________
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

test_loss /= len(testset)
test_acc = 100.0 * correct / total
print(f"\nTest‑loss: {test_loss:.4f}  Test‑acc: {test_acc:.2f}%")


Test‑loss: 1.7221  Test‑acc: 63.81%


## Assigment 2:
Load a **pre-trained** (on ImageNet) "*ResNet18*" from the torchvision model zoo and *fine-tune* it for ten epochs

In [7]:
#Transform‑Pipeline – auf 224×224 skalieren, weil vortrainiert, erwarten cnn filter  224px werte! wichtig, sonst sind ergebnisse nicht räpresentativ!
transform_ft = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

#DataLoader neu instanziieren
trainset_ft = torchvision.datasets.CIFAR10("./data", train=True,  download=False, transform=transform_ft)
testset_ft  = torchvision.datasets.CIFAR10("./data", train=False, download=False, transform=transform_ft)

trainloader_ft = torch.utils.data.DataLoader(trainset_ft, batch_size=batch_size,
                                             shuffle=True,  num_workers=2, pin_memory=True)
testloader_ft  = torch.utils.data.DataLoader(testset_ft,  batch_size=batch_size,
                                             shuffle=False, num_workers=2, pin_memory=True)


In [8]:

#Pre‑trained ResNet‑18 -- nicht die default weights wie ass.1
model_ft = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
model_ft.fc = torch.nn.Linear(model_ft.fc.in_features, 10)  # CIFAR‑10 --> 10 Klassen
model_ft = model_ft.to(device)


In [9]:
#Backbone einfrieren, nur den neuen Kopf trainieren
for p in model_ft.parameters():
    p.requires_grad = False
for p in model_ft.fc.parameters():
    p.requires_grad = True


In [10]:

#criterion und Optimizer --> CE, da nicht ordinal und std lr und wd werte, diesmal Adam gewählt
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ft.fc.parameters(), lr=3e-4, weight_decay=1e-4)


In [11]:
#______________________________
#Training (10 epochs)
#______________________________
epochs = 10
for epoch in range(epochs):
    model_ft.train()
    tot_loss, tot_correct, tot_samples = 0.0, 0, 0

    for x, y in trainloader_ft:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits = model_ft(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        tot_loss += loss.item() * x.size(0)
        tot_correct += logits.argmax(1).eq(y).sum().item()
        tot_samples += y.size(0)

    print(f"[{epoch+1:02d}/{epochs}] "
          f"train_loss={tot_loss/tot_samples:.4f}  "
          f"train_acc={100*tot_correct/tot_samples:.2f}%")




[01/10] train_loss=1.1658  train_acc=61.27%
[02/10] train_loss=0.9938  train_acc=66.82%
[03/10] train_loss=0.9804  train_acc=67.22%
[04/10] train_loss=0.9711  train_acc=67.62%
[05/10] train_loss=0.9635  train_acc=67.67%
[06/10] train_loss=0.9534  train_acc=68.14%
[07/10] train_loss=0.9580  train_acc=68.05%
[08/10] train_loss=0.9536  train_acc=68.18%
[09/10] train_loss=0.9475  train_acc=68.58%
[10/10] train_loss=0.9447  train_acc=68.48%


In [12]:
#______________________________________________________
# Eval
#_______________________________________________________
model_ft.eval()
test_loss, test_correct, test_total = 0.0, 0, 0
with torch.no_grad():
    for x, y in testloader_ft:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        logits = model_ft(x)
        test_loss += criterion(logits, y).item() * x.size(0)
        test_correct += logits.argmax(1).eq(y).sum().item()
        test_total += y.size(0)

print(f"\nTest_loss={test_loss/test_total:.4f}  Test_acc={100*test_correct/test_total:.2f}%")



Test_loss=0.6421  Test_acc=78.27%


Wie erwartet ist das Ergebnis aus ass2 mit dem pretrained gewcihten etwas besser (ich persönlich dachte es wäre deutlich besser)
Ass1 zeigt allerdings einen leichten overfit und scheint nicht optimal zu generalisieren

Ass2 scheint zumindest bei diesem sample sehr gute testergebnisse zu liefern. Leider ist wie unten nochmals erwähnt der vollständige test ohne backbone nicht möglich (wollte ich interessenshalber mal testen) um weitere rückschlüsse zu ziehen




Mögliche verbesserungen:
besseres lr rrate scheduling (cosin cyclic learning rate)
data augmentation (verschiedene bildausschnitte oder bilder drehen usw)

________________________________________________________________________________


Hier ohne backbone und fulltest:


In [13]:
#Transform‑Pipeline – auf 224×224 skalieren, weil vortrainiert, erwarten cnn filter  224px werte! wichtig, sonst sind ergebnisse nicht räpresentativ!
transform_ft = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

#DataLoader neu instanziieren
trainset_ft = torchvision.datasets.CIFAR10("./data", train=True,  download=False, transform=transform_ft)
testset_ft  = torchvision.datasets.CIFAR10("./data", train=False, download=False, transform=transform_ft)

trainloader_ft = torch.utils.data.DataLoader(trainset_ft, batch_size=batch_size,
                                             shuffle=True,  num_workers=2, pin_memory=True)
testloader_ft  = torch.utils.data.DataLoader(testset_ft,  batch_size=batch_size,
                                             shuffle=False, num_workers=2, pin_memory=True)


In [14]:

#Pre‑trained ResNet‑18 -- nicht die default weights wie ass.1
model_ft = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
model_ft.fc = torch.nn.Linear(model_ft.fc.in_features, 10)  # CIFAR‑10 --> 10 Klassen
model_ft = model_ft.to(device)


In [15]:

#criterion und Optimizer --> CE, da nicht ordinal und std lr und wd werte, diesmal Adam gewählt
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ft.fc.parameters(), lr=3e-4, weight_decay=1e-4)


In [ ]:
#______________________________
#Training (10 epochs)
#______________________________
epochs = 10
for epoch in range(epochs):
    model_ft.train()
    tot_loss, tot_correct, tot_samples = 0.0, 0, 0

    for x, y in trainloader_ft:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits = model_ft(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        tot_loss += loss.item() * x.size(0)
        tot_correct += logits.argmax(1).eq(y).sum().item()
        tot_samples += y.size(0)

    print(f"[{epoch+1:02d}/{epochs}] "
          f"train_loss={tot_loss/tot_samples:.4f}  "
          f"train_acc={100*tot_correct/tot_samples:.2f}%")




[01/10] train_loss=1.1699  train_acc=61.06%
[02/10] train_loss=1.0016  train_acc=66.29%
[03/10] train_loss=0.9736  train_acc=67.50%
[04/10] train_loss=0.9722  train_acc=67.77%


In [ ]:
#______________________________________________________
# Eval
#_______________________________________________________
model_ft.eval()
test_loss, test_correct, test_total = 0.0, 0, 0
with torch.no_grad():
    for x, y in testloader_ft:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        logits = model_ft(x)
        test_loss += criterion(logits, y).item() * x.size(0)
        test_correct += logits.argmax(1).eq(y).sum().item()
        test_total += y.size(0)

print(f"\nTest_loss={test_loss/test_total:.4f}  Test_acc={100*test_correct/test_total:.2f}%")


leider habe ich die gpu nutzung überschritten, weshalb der letzte test nicht vollständig evaluiert werden kann...